<a href="https://colab.research.google.com/github/AdrianDiez/Maingear-Estimator/blob/main/Maingear_Estimates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Maingear Estimator

Given the low availability of data after some research it was found that using logarithmic regression would be the most addecuate given the simplicity of the task and the amount of information in the dataset.
Source: [The Best Classifier for Small Datasets: Log-F(m,m) Logit](https://medium.com/@remycanario17/log-f-m-m-logit-the-best-classification-algorithm-for-small-datasets-fc92fd95bc58)

## Dependencies
The first step is to gather all necesary dependencies, in this case we would be reading from a Google Spreadsheet and converting the information as needed.

Documentation: [Read from sheets](https://developers.google.com/sheets/api/quickstart/python)

In [ ]:
pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

     |████████████████████████████████| 7.0MB 9.1MB/s 
Requirement already up-to-date: google-auth-oauthlib in /usr/local/lib/python3.7/dist-packages (0.4.4)
ERROR: earthengine-api 0.1.260 has requirement google-api-python-client<2,>=1.12.1, but you'll have google-api-python-client 2.2.0 which is incompatible.
  Found existing installation: google-auth-httplib2 0.0.4
    Uninstalling google-auth-httplib2-0.0.4:
      Successfully uninstalled google-auth-httplib2-0.0.4
  Found existing installation: google-api-python-client 1.12.8
    Uninstalling google-api-python-client-1.12.8:
      Successfully uninstalled google-api-python-client-1.12.8


In [30]:
import pandas as pd
import numpy as np
import gspread

from google.colab import auth
from oauth2client.client import GoogleCredentials

In [92]:
dataset_url = 'https://docs.google.com/spreadsheets/d/1z6sD5_iGArHKal-hdd2BZkHuPhRJ47Xfu3ET1D0fj0E/edit?ts=60463182#gid=1539062128' 
tab_name = 'All Builds'
critical_columns = ['RIG', 'APEX', 'BUILD', 'CPU', 'GPU']
column_name = {'APEX?' : 'APEX', 'Rig' : 'RIG', 'Build' : 'BUILD', 'User' : 'USER','Assembled ?' : 'COMPLETED', 
               'Paint?' : 'PAINT', 'Ordered Date' : 'ORDERED_DATE', 'Completed Date' : 'COMPLETED_DATE', 'Days' : 'DAYS'}
#if the dataset is moved but the structure is the same this could be pointed to other links or other tabs.

In [130]:
auth.authenticate_user() 
# This step will ask you to go into a link and approve the access to this tool. Don't do it if you don't feel confortable. You can search in internet, this is standard from Google.

gc = gspread.authorize(GoogleCredentials.get_application_default())
wb = gc.open_by_url(dataset_url) 
sheet = wb.worksheet(tab_name) 

data = sheet.get_all_values()[2:] # We skip the first two rows, no data 
orig_df = pd.DataFrame(data[1:]) # Skipping the header
orig_df.columns = data[0] # Setting up the header
orig_df = orig_df.rename(columns=column_name)

orig_df = orig_df.applymap(lambda s:s.lower() if type(s) == str else s) # All to lowercase so I don't go crazy
orig_df = orig_df.applymap(lambda s:s.rstrip('?') if type(s) == str else s) # Removing ? just in case

clean_df = orig_df.replace(r'^\s*$', np.nan, regex=True).dropna(subset=critical_columns) # Converting blanks to NaN and droping rows with NaN in critical values. (see above)
clean_df['USER'] = clean_df['USER'].apply(hash) # Hashing Usernames :)
clean_df['GPU'] = clean_df['GPU'].map(lambda x: x.rstrip('x2').strip()).to_list() # Removing dual 3090, residual population
clean_df['DAYS'] = clean_df['DAYS'].astype(int)
###### There are records which are dropped because of NaN. !!!!!!!!!!

In [131]:
mask = clean_df['COMPLETED'] == 'true'
completed_df = clean_df[mask]
not_completed_df = clean_df[~mask]

In [132]:
completed_df['GPU'].value_counts()

3080       52
3090       17
3060 ti    12
3070        9
6800xt      3
2060        3
Name: GPU, dtype: int64

In [157]:
avg_dict = {}

for col in critical_columns:
  sup_dict = {}
  avg_dict[col] = sup_dict
  for val in completed_df[col].unique():
    mask = completed_df[col] == val
    sup_dict[val] = round(completed_df[mask]['DAYS'].mean())

In [162]:
avg_dict['CPU']

{'10900k': 98,
 '3970x': 127,
 '5600x': 53,
 '5800x': 74,
 '5900x': 75,
 '5900x ': 57,
 '5950x': 96}

In [129]:
completed_df['DAYS'].astype(int).mean()

76.48958333333333

In [147]:
gpu_dict

{'2060': 81,
 '3060 ti': 50,
 '3070': 59,
 '3080': 74,
 '3090': 110,
 '6800xt': 87,
 str: int}